# Model Formulation

This notebook can be used to run initial model with DCOPF and a basic budget variable. Before running this nodebook please refer to `Test-System-Data-Processing.ipynb` to preprocess generator, lines, buses, and loads data. 

In [1]:
using JuMP, Gurobi
using DataFrames, CSV

In [127]:
# Global path to test system datasets
PATH = joinpath(pwd(), "..", "data", "test_system") 

# Load each of the San Diego test system files 
gens = CSV.read(joinpath(PATH, "gens.csv"), DataFrame);
lines = CSV.read(joinpath(PATH, "lines.csv"), DataFrame);
buses = CSV.read(joinpath(PATH, "buses.csv"), DataFrame);
loads = CSV.read(joinpath(PATH, "loads_24h_sp.csv"), DataFrame);

gens = gens[gens[:, :fueltype] .≠ "Synchronous Condenser", :]

# Add load to bus dataset equal to first hour of load
buses[:, :pd] = loads[:, 2];
println("Consistent buses: ", all(buses[:, :bus] .== loads[:, :bus]))

# Add line susceptance B = X / (R² + X²)
lines[:, :sus] = lines.x ./ (lines.r .^2 + lines.x .^2);

Consistent buses: true


In [129]:
function transport_model(gens, lines, buses)
    """
    Function to solve DC OPF problem 
    Inputs:
        gens -- dataframe with generator info and costs
        lines -- dataframe with transmission lines info
        buses -- dataframe with bus types and loads
    """
    # Sets
    G = 1:nrow(gens)       # G: Set of all generators
    N = 1:nrow(buses)      # N: Set of all network nodes
    L = 1:nrow(lines)      # L: Set of all lines/branches
    fbus = lines[!,:f_bus] # All from buses
    tbus = lines[!,:t_bus] # All to buses    
    demand = buses[!,:pd]  # Demand at bus all buses

    # Parameters
    shed_cost = 1000 # Cost of load sheding
    baseMVA = 100
    
    # Model
    DCOPF = Model(()->Gurobi.Optimizer())
    set_optimizer_attribute(DCOPF, "NumericFocus", 2)
    
    # Decision variables   
    @variables(DCOPF, begin
        GEN[G]          # Generation of each generator 
        SHED[N] ≥ 0     # Load sheading at bus N
        FLOW[N,N]       # Flows between all pairs of nodes
    end)
                
    # Objective function (Note: Using just the linear part of quadratic cost)
    @objective(DCOPF, Min, 
        sum(gens[g,:c1] * GEN[g] for g ∈ G) + sum(shed_cost*SHED[i] for i ∈ N))

    @constraint(DCOPF, cShed[i ∈ N], SHED[i] ≤ demand[i])
    
    # Supply demand balances: sum(generation) + shedding - demand = sum(flows)
    @constraint(DCOPF, cBalance[i ∈ N], 
        sum(GEN[g] for g ∈ G[gens.bus .== i])
        - demand[i] + SHED[i]
         == sum(FLOW[i,j] for j ∈ fbus[tbus .== i]))

    # Max and min generation constraints
    @constraint(DCOPF, cMaxGen[g ∈ G], GEN[g] ≤ gens[g,:pmax])
    @constraint(DCOPF, cMinGen[g ∈ G], GEN[g] ≥ gens[g,:pmin])
    
    # Max line flow constraints
    @constraint(DCOPF, cLineLimits[l ∈ L], 
        FLOW[lines[l,:f_bus], lines[l,:t_bus]] ≤ lines[l,:rate_a] * baseMVA)  
    
    # Anti-symmetric flow constraints
    @constraint(DCOPF, cFlowSymmetric[i in N, j in N],
                    FLOW[i,j] == -FLOW[j,i])
    
    return DCOPF
end

transport_model (generic function with 1 method)

In [131]:
function model_outputs(model, gens, lines, buses)

    N = 1:nrow(buses)      # N: Set of all network nodes
    fbus = lines[!,:f_bus] # All from buses
    tbus = lines[!,:t_bus] # All to buses
    baseMVA = 100
    
    generation = DataFrame(
        id = gens.id,
        node = gens.bus,
        gen = value.(model[:GEN]).data
        )
    
    # Angles of each bus
    #angles = value.(model[:THETA]).data
    
    flows = DataFrame(
        fbus = fbus,
        tbus = tbus,
        #flow = value.(model[:FLOW]).data #baseMVA .* lines.sus .* (angles[fbus] .- angles[tbus])
    )
    
    prices = DataFrame(
        node = N,
        value = dual.(model[:cBalance]).data
    )
    # Amound of load shed at each bus
    shedding = DataFrame(
        node = N,
        shed =value.(model[:SHED]).data
    )
    
    return (
        generation = generation, 
        #angles,
        flows,
        prices,
        shedding,
        cost = objective_value(model),
        status = termination_status(model)
    )
end

model_outputs (generic function with 1 method)

In [133]:
dcopf = transport_model(gens, lines, buses)

#dcopf = initialize_model(gens, lines, buses)

Set parameter Username
Set parameter LicenseID to value 2669913
Academic license - for non-commercial use only - expires 2026-05-22
Set parameter NumericFocus to value 2


A JuMP Model
├ solver: Gurobi
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 239754
├ num_constraints: 242102
│ ├ AffExpr in MOI.EqualTo{Float64}: 239610
│ ├ AffExpr in MOI.GreaterThan{Float64}: 144
│ ├ AffExpr in MOI.LessThan{Float64}: 1859
│ └ VariableRef in MOI.GreaterThan{Float64}: 489
└ Names registered in the model
  └ :FLOW, :GEN, :SHED, :cBalance, :cFlowSymmetric, :cLineLimits, :cMaxGen, :cMinGen, :cShed

In [135]:
optimize!(dcopf)

Set parameter NumericFocus to value 2
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2 Ultra
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Non-default parameters:
NumericFocus  2

Optimize a model with 241613 rows, 239754 columns and 481601 nonzeros
Model fingerprint: 0xe7b2ec6e
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [7e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-03, 3e+03]
Presolve removed 241247 rows and 239017 columns
Presolve time: 0.10s
Presolved: 366 rows, 737 columns, 1217 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 4.800e+02
 Factor NZ  : 3.868e+03
 Factor Ops : 4.674e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual       

In [137]:
solution = model_outputs(dcopf, gens, lines, buses);

In [139]:
# Get generation by type
gdf = groupby(innerjoin(gens, solution.generation, on = :id), :fueltype)
df = combine(gdf, :gen => sum)
df[:, :max_gen] .= combine(groupby(gens, :fueltype), :pmax => sum)[:, :pmax_sum]
df

Row,fueltype,gen_sum,max_gen
,String,Float64,Float64
1,Conventional Hydroelectric,0.0,8.9
2,Other Waste Biomass,0.0,12.3
3,Natural Gas Fired Combustion Turbine,73.2,1656.3
4,Petroleum Liquids,0.0,4.5
5,Landfill Gas,5.4,17.0
6,Natural Gas Fired Combined Cycle,519.692,1346.2
7,Solar Photovoltaic,684.085,684.085
8,Natural Gas Internal Combustion Engine,3.6,3.6
9,Onshore Wind Turbine,194.0,194.0


In [141]:
sum(solution.shedding[:, :shed]) / sum(buses[:, :pd])

0.0142060444154652

In [53]:


CSV.write("shed.csv", solution.shedding)

"shed.csv"

In [155]:
maximum(lines[:, :rate_a])*100

1464.0

In [309]:
function initialize_model(gens, lines, buses)
    """
    Function to solve DC OPF problem 
    Inputs:
        gens -- dataframe with generator info and costs
        lines -- dataframe with transmission lines info
        buses -- dataframe with bus types and loads
    """
    # Sets
    G = 1:nrow(gens)       # G: Set of all generators
    N = 1:nrow(buses)      # N: Set of all network nodes
    L = 1:nrow(lines)      # L: Set of all lines/branches
    fbus = lines[!,:f_bus] # All from buses
    tbus = lines[!,:t_bus] # All to buses    
    demand = buses[!,:pd]  # Demand at bus all buses

    # Parameters
    baseMVA = 100      # The base MVA is 100 MVA
    θlim = π*(30/180)  # Absolute max angle limit in rad
    slack_bus = 416    # Set slack bus to one with many CTs
    shed_cost = 1000   # Cost of load sheding

    # Toy parameters for a limited investment in solar at each bus
    budget = 0
    solar_cost = 0.1475
    
    # Model
    DCOPF = Model(()->Gurobi.Optimizer())
    set_optimizer_attribute(DCOPF, "NumericFocus", 2)
    
    # Decision variables   
    @variables(DCOPF, begin
        GEN[G]          # Generation of each generator 
        SHED[N] ≥ 0     # Load sheading at bus N
        THETA[N]        # Voltage phase angle of bus
        FLOW[N,N]       # Flows between all pairs of nodes
        
    end)
                
    # Objective function (Note: Using just the linear part of quadratic cost)
    @objective(DCOPF, Min, 
        sum(gens[g,:c1] * GEN[g] for g ∈ G) + sum(shed_cost*SHED[i] for i ∈ N))
    
    # Supply demand balances: sum(generation) + shedding - demand - 0.5*sum(losses) = sum(flows)
    @constraint(DCOPF, cBalance[i ∈ N], 
        sum(GEN[g] for g ∈ G[gens.bus .== i])
        - demand[i] + SHED[i]
         == sum(FLOW[i,j] for j ∈ tbus[fbus .== i]))
    
    # Max and min generation constraints
    @constraint(DCOPF, cMaxGen[g ∈ G], GEN[g] ≤ gens[g,:pmax])
    @constraint(DCOPF, cMinGen[g ∈ G], GEN[g] ≥ gens[g,:pmin])
    
    # Max line flow constraints
    @constraint(DCOPF, cLineLimits[l ∈ L], 
        FLOW[lines[l,:f_bus], lines[l,:t_bus]] ≤ lines[l,:rate_a]*baseMVA) 

    # Create slack bus with theta=0
    fix(THETA[slack_bus], 0)
    
    # Angle limits 
    #@constraint(DCOPF, cAngleLimitsMax[l ∈ L], 
    #    (THETA[fbus[l]] - THETA[tbus[l]]) ≤  θlim)
    #@constraint(DCOPF, cAngleLimitsMin[l ∈ L], 
    #    (THETA[fbus[l]] - THETA[tbus[l]]) ≥ -θlim)
                    
    # Flow constraints on each branch
    @constraint(DCOPF, cLineFlows[l ∈ L],
            FLOW[fbus[l], tbus[l]] ==
            baseMVA*lines[l,:sus]*(THETA[fbus[l]] - THETA[tbus[l]]))
    
    return DCOPF
end

initialize_model (generic function with 1 method)

In [368]:
lines[!,:rate_a]*100

1226-element Vector{Float64}:
  191.0
   64.0
   56.99999999999999
   64.0
   64.0
   64.0
 1056.0
  869.9999999999999
 1158.0
 1158.0
  250.0
 2811.0
 1462.0
    ⋮
  129.9168355
   79.0
   73.099625
   46.88339725
   22.0
  194.7702155
  101.0
  493.0
  529.0
   20.0
  101.5685185
  193.5590785